In [1]:
import pandas as pd
import numpy as np
import tensorflow
import theano
import pickle

from pandas import *

from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.metrics import *

from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM
from keras.wrappers.scikit_learn import KerasClassifier
from keras import backend

import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.externals.joblib import parallel_backend
from random import *

Using TensorFlow backend.


In [2]:
df = pd.read_csv('../data/df_for_model.csv')
# df['start_time'] = df['start_time']//1000
df = df[df['result'] != 0.5]

In [3]:
bins = list(range(-2000, -499, 50))
bins.extend(list(range(-490, -199, 10)))
bins.extend(list(range(-195, 196, 5)))
bins.extend(list(range(200, 501, 10)))
bins.extend(list(range(550, 2001, 50)))

In [4]:
df['diff_bin'] = pd.cut(df['diff'], bins=bins, labels=bins[:len(bins)-1])

In [5]:
numeric_predictors = ['color', 'diff_bin', 'game_time', 'start_time', 'weekday']
# X = df[numeric_predictors].values
X = df[numeric_predictors]
y = np.array(df['result'])

In [6]:
X.shape

(2076, 5)

In [7]:
y.shape

(2076,)

In [8]:
X = X.astype('float64')
y = y.astype('int64')

In [9]:
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.15, random_state=6)

In [10]:
limi = randint(int(len(X)*.85), int(len(X)*.9))
limi

1833

In [11]:
X_train = X[:limi]
y_train = y[:limi]
X_test = X[limi:]
y_test = y[limi:]

In [12]:
X_train.shape

(1833, 5)

In [13]:
X_test.shape

(243, 5)

In [14]:
y_train.shape

(1833,)

In [15]:
std_sclr = StandardScaler()
X_train = std_sclr.fit_transform(X_train)
X_test = std_sclr.fit_transform(X_test)

In [16]:
# std_sclr.scale_

In [17]:
# def _classifier():
#     classifier = Sequential()

#     classifier.add(Dense(units=13, kernel_initializer='uniform',
#                          activation='relu', input_dim=5))

#     classifier.add(
#         Dense(units=83, kernel_initializer='uniform', activation='softmax'))

#     classifier.add(
#         Dense(units=13, kernel_initializer='uniform', activation='relu'))

# #     classifier.add(
# #         Dense(units=109, kernel_initializer='uniform', activation='softmax'))

# #     classifier.add(
# #         Dense(units=47, kernel_initializer='uniform', activation='relu'))

# #     classifier.add(
# #         Dense(units=11, kernel_initializer='uniform', activation='softmax'))

#     classifier.add(
#         Dense(units=1, kernel_initializer='uniform', activation='sigmoid'))

#     classifier.compile(
#         optimizer='sgd', loss='binary_crossentropy', metrics=['hinge'])

#     return classifier

In [18]:
def dropout_classifier(optimizer, metrics):
    classifier = Sequential()
    classifier.add(Dense(units=5, kernel_initializer='uniform',
                         activation='relu', input_dim=5))
    classifier.add(Dropout(rate=0.16))
    classifier.add(
        Dense(units=23, kernel_initializer='uniform', activation='softmax'))
    classifier.add(Dropout(rate=0.08))
    classifier.add(
        Dense(units=8, kernel_initializer='uniform', activation='relu'))
    classifier.add(Dropout(rate=0.04))
    classifier.add(
        Dense(units=1, kernel_initializer='uniform', activation='tanh'))

    classifier.compile(
        optimizer=optimizer, loss='binary_crossentropy', metrics=[metrics])
    return classifier

In [20]:
bs_1 = randint(10,20)
bs_2 = randint(20,30)
ep_1 = randint(100,200)
ep_2 = randint(200,300)

parameters = {'batch_size': [bs_1, bs_2],
              'nb_epoch': [ep_1, ep_2],
              'optimizer': ['Nadam', 'SGD', 'RMSprop'],
              'metrics': ['accuracy', 'hinge', 'mape']
              }

In [21]:
classifier = KerasClassifier(build_fn=dropout_classifier)

In [22]:
grid_search = GridSearchCV(classifier, parameters,
                           scoring='balanced_accuracy', cv=5, n_jobs=-2)

In [ ]:
print([bs_1, bs_2, ep_1, ep_2])
grid_search = grid_search.fit(X=X_train, y=y_train, shuffle=False)

Epoch 1/1
1466/1466 [==============================] - 1s 661us/step - loss: 1.8553 - acc: 0.5007
Epoch 1/1
1466/1466 [==============================] - 1s 661us/step - loss: 2.5490 - acc: 0.5089
Epoch 1/1
1466/1466 [==============================] - 1s 702us/step - loss: 7.9711 - acc: 0.5055
Epoch 1/1
1467/1467 [==============================] - 1s 702us/step - loss: 2.0759 - acc: 0.5003
Epoch 1/1
1467/1467 [==============================] - 1s 746us/step - loss: 2.1653 - acc: 0.5078
Epoch 1/1
1466/1466 [==============================] - 1s 659us/step - loss: 7.9615 - acc: 0.4986
Epoch 1/1
1466/1466 [==============================] - 1s 685us/step - loss: 8.0395 - acc: 0.4932
Epoch 1/1
1466/1466 [==============================] - 1s 647us/step - loss: 4.1472 - acc: 0.4966
Epoch 1/1
1467/1467 [==============================] - 1s 669us/step - loss: 7.9176 - acc: 0.5017
Epoch 1/1
1467/1467 [==============================] - 1s 711us/step - loss: 6.4868 - acc: 0.4942
Epoch 1/1
1466/1466 

1467/1467 [==============================] - 4s 3ms/step - loss: 5.1240 - mean_absolute_percentage_error: 7884136.1464
Epoch 1/1
1467/1467 [==============================] - 4s 3ms/step - loss: 1.8387 - mean_absolute_percentage_error: 21317228.1576
Epoch 1/1
1466/1466 [==============================] - 4s 2ms/step - loss: 8.0481 - mean_absolute_percentage_error: 1080170.4415
Epoch 1/1
1466/1466 [==============================] - 4s 2ms/step - loss: 7.9161 - mean_absolute_percentage_error: 728113.6880
Epoch 1/1
1466/1466 [==============================] - 4s 3ms/step - loss: 8.0201 - mean_absolute_percentage_error: 497954416.8637
Epoch 1/1
1467/1467 [==============================] - 4s 3ms/step - loss: 7.9021 - mean_absolute_percentage_error: 492844682.9389
Epoch 1/1
1467/1467 [==============================] - 4s 3ms/step - loss: 7.9327 - mean_absolute_percentage_error: 574191.8967
Epoch 1/1
1466/1466 [==============================] - 4s 3ms/step - loss: 1.9521 - mean_absolute_percen

In [ ]:
best_params = grid_search.best_params_
best_params

In [ ]:
best_accu = grid_search.best_score_
best_accu

In [ ]:
grid_search.cv_results_

In [ ]:
y_pred = grid_search.predict(X_test)
y_pred = (y_pred > 0.5)

In [ ]:
cm = confusion_matrix(y_test, y_pred)
cm

In [ ]:
print(f'classification_report:\n{classification_report(y_test, y_pred)}')
print('---------------------------------------------------------------\n\n')
print(f'average_precision_score:\n{average_precision_score(y_test, y_pred)}')
print('---------------------------------------------------------------\n\n')
print(f'f1_score:\n{f1_score(y_test, y_pred)}')
print('---------------------------------------------------------------\n\n')
print(f'accuracy_score:\n{accuracy_score(y_test, y_pred)}')
print('---------------------------------------------------------------\n\n')
print(f'cohen_kappa_score:\n{cohen_kappa_score(y_test, y_pred)}')
print('---------------------------------------------------------------\n\n')
print(f'roc_curve:\n{roc_curve(y_test, y_pred)}')
print('---------------------------------------------------------------\n\n')
print(f'roc_auc_score:\n{roc_auc_score(y_test, y_pred)}')
print('---------------------------------------------------------------\n\n')
print(f'brier_score_loss:\n{brier_score_loss(y_test, y_pred)}')
print('---------------------------------------------------------------\n\n')
print(recall_score(y_test, y_pred))

In [ ]:
print(f'Accuracy = {round((cm[0][0]+cm[1][1])/(cm.sum())/.01,1)}%')

In [ ]:
plt.plot(grid_search.grid_search['acc'])
plt.plot(grid_search.grid_search['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(grid_search.grid_search['loss'])
plt.plot(grid_search.grid_search['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
def dropout_classifier_set():
    classifier = Sequential()
    classifier.add(Dense(units=5, kernel_initializer='uniform',
                         activation='relu', input_dim=5))
    classifier.add(Dropout(rate=0.12))
    classifier.add(
        Dense(units=40, kernel_initializer='uniform', activation='relu'))
    classifier.add(Dropout(rate=0.08))
    classifier.add(
        Dense(units=8, kernel_initializer='uniform', activation='relu'))
    classifier.add(Dropout(rate=0.04))
    classifier.add(
        Dense(units=1, kernel_initializer='uniform', activation='sigmoid'))

    classifier.compile(
        optimizer='Nadam', loss='binary_crossentropy', metrics=['accuracy'])
    return classifier

In [ ]:
# classifier = dropout_classifier()

In [ ]:
classifier.fit(X_train, y_train, batch_size=13, 
                   epochs=250, shuffle=False)

In [ ]:
batch_list = [134, 67, 28, 14, 7, 4]
for num in batch_list:
    classifier.fit(X_train, y_train, batch_size=num, 
                   epochs=100, shuffle=False)

In [ ]:
y_pred = classifier.predict(X_test)
y_pred = (y_pred > 0.5)

In [ ]:
cm = confusion_matrix(y_test, y_pred)
cm

In [ ]:
print(f'Accuracy = {(cm[0][0]+cm[1][1])/(cm.sum())/.01}%')

### 75.5% Accuracy on 1824 training, 200 test
__3 layers__
- layer 1 = ReLU X 7
- layer 2 = SoftMax X 14
- layer 3 = ReLU X 7
- output layer = Sigmoid X 1

__metrics__ = Cosine

__loss__ = binary_crossentropy

__Optimizer__ = Nadam

__Batch size__ = 12

__Epochs__ = 175

no __diff__

In [ ]:
classifier = KerasClassifier(build_fn=_classifier, batch_size=12, epochs=200)

In [ ]:
accus = cross_val_score(estimator=classifier, X=X_train,
                        y=y_train, cv=10, n_jobs=-2, scoring='accuracy')
accus

In [ ]:
accu_mean = accus.mean()
accu_mean

In [ ]:
accu_var = accus.std()
accu_var

In [ ]:
accus

In [ ]:
def dropout_classifier(optimizer, metrics):
    classifier = Sequential()
    classifier.add(Dense(units=5, kernel_initializer='uniform',
                         activation='relu', input_dim=5))
    classifier.add(Dropout(rate=0.12))
    classifier.add(
        Dense(units=40, kernel_initializer='uniform', activation='relu'))
    classifier.add(Dropout(rate=0.08))
    classifier.add(
        Dense(units=8, kernel_initializer='uniform', activation='relu'))
    classifier.add(Dropout(rate=0.04))
    classifier.add(
        Dense(units=1, kernel_initializer='uniform', activation='sigmoid'))

    classifier.compile(
        optimizer=optimizer, loss='binary_crossentropy', metrics=[metrics])
    return classifier